In [1]:
## todo: 
# 解决测评函数不能引入cv的问题 done
# 建立cv流程进行调参 在模型处引入权重因素
# 清理异常点(小于0) done

#训练集的最优是过拟合的结果，选择次优方案

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_test=pd.read_csv('input/train_time.csv')

In [4]:
train_test=train_test.fillna(-1)

In [5]:
train_test.head()

,id,label,college,rank,total_people,rank_percent,countM1,price_sumM1,price_avgM1,price_maxM1,...,地点263_avg,地点263_max,地点263_min,地点263_median,地点840_count,地点840_sum,地点840_avg,地点840_max,地点840_min,地点840_median
0,0,0.0,9.0,1.0,2933.0,0.000341,49.0,201.31,4.108367,36.4,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
1,1,0.0,9.0,2.0,2933.0,0.000682,-1.0,-1.00,-1.000000,-1.0,...,-1.0,-1.0,-1.0,-1.0,19.0,117.0,6.157895,7.0,4.0,6.0
2,8,0.0,6.0,1565.0,1572.0,0.995547,97.0,347.74,3.584948,10.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
3,9,0.0,6.0,1570.0,1572.0,0.998728,98.0,491.01,5.010306,17.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
4,10,0.0,3.0,1.0,2301.0,0.000435,27.0,82.96,3.072593,22.3,...,-1.0,-1.0,-1.0,-1.0,2.0,7.0,3.500000,4.0,3.0,3.5


In [6]:
## Evaluation 
def f1_macro(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        print '\n%d'%i
        if sum(df['subsidy_x'] == i)!=0:
            r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
            print 'Recall---%s'%r
        else: 
            r=0
        if sum(df['subsidy_y'] == i)!=0:
            p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
            print 'Precision---%s'%p
        else:
            p=0
        if (r+p)!=0:
            f = r*p*2/(r+p)
            print 'F1---%s'%f
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    
    print '\nF1-macro---%s'%s
    return s

def f1_macro0_drop(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        if sum(df['subsidy_x'] == i)!=0:
            r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
        else: 
            r=0
        if sum(df['subsidy_y'] == i)!=0:
            p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
        else:
            p=0
            
        if (r+p)!=0:
            f = r*p*2/(r+p)
        else:
            f=0
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    
    return s

In [24]:
nice_feature=pd.read_csv('input/nice_feature.csv',header=None,index_col=0)
feature_imp_place20=pd.read_csv('input/feature_imp_place20.csv')
#big_nice_feature=pd.read_csv('input/big_nice_feature.csv')

target = 'label'
IDcol = 'id'

all_feature = [x for x in train_test.columns if x not in [target,IDcol]]

predictors=all_feature
#predictors = [ x for x in all_feature if (x in nice_feature.index)|(x in feature_imp_place20.feature.values)]

In [25]:
X = train_test.ix[:, train_test.columns != 'label']
y = train_test.ix[:, train_test.columns == 'label']

In [26]:
from sklearn.cross_validation import KFold

In [27]:
rng = np.random.RandomState(42)
kf = KFold(y.shape[0], n_folds=3, shuffle=True,random_state=rng)

In [28]:
#skf=StratifiedKFold(z,n_folds=3,shuffle=True,random_state=rng)

In [34]:
def model_tuning():
    
#for tuning_target in [1]:
    score=[]
    for train_index, test_index in kf:
        train,train['label']=X.iloc[train_index],y.iloc[train_index]
    
        Oversampling1000 = train.loc[train.label == 1000]
        Oversampling1500 = train.loc[train.label == 1500]
        Oversampling2000 = train.loc[train.label == 2000]
        for i in range(a):
            train = train.append(Oversampling1000)
        for j in range(b):
            train = train.append(Oversampling1500)
        for k in range(c):
            train = train.append(Oversampling2000)
        
        param_dist = {
            'n_estimators': 120,
            'max_depth': 7,
            'min_child_weight':7,
            'gamma':0,
            'subsample':0.4,
            'colsample_bytree':1,
            'reg_alpha':0.1,
            'learning_rate':0.05}
        
        '''
          param_dist = {
            'n_estimators': 60,
            'max_depth': 5,
            'learning_rate':0.2,
            'min_child_weight':2,
            'gamma':0,
            'subsample':1,
            'colsample_bytree':1,
            'reg_alpha':0.008}
        '''      
        xgb_model = XGBClassifier(**param_dist).fit(train[predictors],train[target])
        predict = xgb_model.predict(X.iloc[test_index][predictors])
        
        true = y.iloc[test_index]
        print f1_macro0_drop(true,predict)
        score.append(f1_macro0_drop(true,predict))
    #print tuning_target,
    print pd.DataFrame(score).mean()

In [32]:
param_dist = {
    'n_estimators': 60,
    'max_depth': 5,
    'learning_rate':0.2,
    'min_child_weight':2,
    'gamma':0,
    'subsample':1,
    'colsample_bytree':1,
    'reg_alpha':0.008}

In [35]:
param_dist={}

In [36]:
#大于线上成绩的模型都可采用，下面的参数都可并入集成子模型
for a in [7,8]:
    for b in [7,8,10]:
        for c in [9,10,11]:
            print a,b,c
            model_tuning()

7 7 9


/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.026782462924
0.0300717951332
0.0231621896966
0    0.026672
dtype: float64
7 7 10
0.0241640922619
0.0295010482095
0.0231596239668
0    0.025608
dtype: float64
7 7 11
0.0211143931036
0.0255566571076
0.0223383298463
0    0.023003
dtype: float64
7 8 9
0.0217725456119
0.0269171827082
0.0230998396852
0    0.02393
dtype: float64
7 8 10
0.0226186385422
0.0283371925543
0.0225196361587
0    0.024492
dtype: float64
7 8 11
0.0263329042129
0.0281134383289
0.0242123380531
0    0.02622
dtype: float64
7 10 9
0.026892937461
0.0295451530542
0.0244650841901
0    0.026968
dtype: float64
7 10 10
0.0253996823388
0.0291622706251
0.0218234318643
0    0.025462
dtype: float64
7 10 11
0.0253800368276
0.0279395940109
0.0226335057296
0    0.025318
dtype: float64
8 7 9
0.0254632302751
0.0290193586694
0.0230141449743
0    0.025832
dtype: float64
8 7 10
0.0238592692349
0.0308250561484
0.0247153348616
0    0.026467
dtype: float64
8 7 11
0.0246709572551
0.0298414449095
0.0242591872245
0    0.026257
dtype: float64
8 8

In [ ]:
0.0310752036941
0.0318844076325
0.0312565245932
0    0.031405
dtype: float64
7 7 10
0.0306199421948
0.0300459478888
0.0305127222103
0    0.030393
dtype: float64
7 7 11
0.0307264298572
0.0293045287458
0.0312028275973
0    0.030411
dtype: float64
7 8 9
0.0316870281635
0.0324066603627
0.0319350212627
0    0.03201
dtype: float64
7 8 10
0.0321757374537
0.0308081674416
0.0309890352377
0    0.031324
dtype: float64
7 8 11
0.0317321625774
0.0320268365579
0.0320917017139
0    0.03195
dtype: float64
7 10 9
0.0307964734137
0.0339801180803
0.0337253421053
0    0.032834
dtype: float64
7 10 10
0.0295198193324
0.0320932018262
0.0302967463066
0    0.030637
dtype: float64
7 10 11
0.0303200639973
0.0337892702597
0.0343904289977
0    0.032833
dtype: float64
8 7 9
0.0297409167768
0.0301859502139
0.0302600182185
0    0.030062
dtype: float64
8 7 10
0.0292952632517
0.0334291177054
0.0317325394815
0    0.031486
dtype: float64
8 7 11
0.0318542646738
0.0312225427818
0.028182098376
0    0.03042
dtype: float64
8 8 9
0.031904806337
0.0339781548937
0.0318626459528
0    0.032582
dtype: float64

In [ ]:
0.03+: 555 579 599 779 799 977 979 999

0.031+: 579 588 688 689 699 768 779 789 878 897

all 0.03: 588 599 689 778 779 7810 7109 71010 878 8710 888 81010
    
all 0.031: 588 689 779 7810 7109 71010 71011 878 8710 81010
    

8710

7109

In [38]:
def model_tuning_GBM():
    
#for tuning_target in [1]:
    score=[]
    for train_index, test_index in kf:
        train,train['label']=X.iloc[train_index],y.iloc[train_index]
    
        Oversampling1000 = train.loc[train.label == 1000]
        Oversampling1500 = train.loc[train.label == 1500]
        Oversampling2000 = train.loc[train.label == 2000]
        for i in range(a):
            train = train.append(Oversampling1000)
        for j in range(b):
            train = train.append(Oversampling1500)
        for k in range(c):
            train = train.append(Oversampling2000)
        

        param_dist = {
            'n_estimators': 60,
            'learning_rate':0.1,
            'max_depth': 5,
            'min_samples_split':200
            'min_samples_leaf':30
        }

            
        gb_model = GradientBoostingClassifier(**param_dist).fit(train[predictors],train[target])
        predict = gb_model.predict(X.iloc[test_index][predictors])
        true = y.iloc[test_index]
        print f1_macro0_drop(true,predict)
        score.append(f1_macro0_drop(true,predict))
    #print tuning_target,
    print pd.DataFrame(score).mean()
    
    #基本款0.031609

In [40]:
for min_samples_leaf in range(20,31,3):
    for min_samples_split in range(120,181,20):
        print min_samples_leaf,min_samples_split
        model_tuning_GBM()

20 120


/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.0296351871405
0.0344766293693
0.0317018726146
0    0.031938
dtype: float64
20 140
0.0308879844468
0.0317513333715
0.0304036847572
0    0.031014
dtype: float64
20 160
0.0298885533991
0.0346910077952
0.028275184921
0    0.030952
dtype: float64
20 180
0.0317100176381
0.0329440129865
0.0305723948037
0    0.031742
dtype: float64
23 120
0.0318638744139
0.033556942373
0.0270590851154
0    0.030827
dtype: float64
23 140
0.0304454328629
0.0332689674271
0.0291043926546
0    0.03094
dtype: float64
23 160
0.029764275608
0.0339638609346
0.0272915911394
0    0.03034
dtype: float64
23 180
0.0304404014181
0.0342046445681
0.0280626566921
0    0.030903
dtype: float64
26 120
0.032038297279
0.0334043060478
0.0280919800046
0    0.031178
dtype: float64
26 140
0.0310400654482
0.0339083019306
0.0263186111739
0    0.030422
dtype: float64
26 160
0.030533029595
0.0338980157542
0.0305160483516
0    0.031649
dtype: float64
26 180
0.0308518298194
0.0350676058903
0.02735735858
0    0.031092
dtype: float64
29 120
0

---

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 7)

In [73]:
train=X_train
train['label']=y_train

test=X_test
test['label']=y_test

test.to_csv('input/test_7r07part.csv',index=False)

/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
nice_feature=pd.read_csv('input/nice_feature.csv',header=None,index_col=0)

target = 'label'
IDcol = 'id'
ids = test['id'].values

all_feature = [x for x in train.columns if x not in [target]]

feature_imp_place20=pd.read_csv('input/feature_imp_place20.csv')
predictors = [ x for x in all_feature if (x in nice_feature.index)|(x in feature_imp_place20.feature.values)]

In [13]:
y_train['label'].value_counts()

0.0       6530
1000.0     515
1500.0     335
2000.0     239
Name: label, dtype: int64

In [14]:

# tips：
# 尝试欠采样和smoth采样

# undersample
train.to_csv('input/train_7r07part.csv',index=False)
train=pd.read_csv('input/train_7r07part.csv')

number_records_fraud = len(train[train.label != 0])
fraud_indices = np.array(train[train.label != 0].index)

# Picking the indices of the normal classes
normal_indices = train[train.label == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = train.iloc[under_sample_indices,:]

X_undersample = under_sample_data.ix[:, under_sample_data.columns != 'label']
y_undersample = under_sample_data.ix[:, under_sample_data.columns == 'label']

# Showing ratio
print("Number of normal transactions: ", len(under_sample_data[under_sample_data.label == 0]))
print("Number of fraud transactions: ", len(under_sample_data[under_sample_data.label != 0]))
print("Total number of transactions in resampled data: ", len(under_sample_data))


# Oversample
Oversampling1000 = train.loc[train.label == 1000]
Oversampling1500 = train.loc[train.label == 1500]
Oversampling2000 = train.loc[train.label == 2000]
'''
for i in range(6):
    train = train.append(Oversampling1000)
for j in range(10):
    train = train.append(Oversampling1500)
for k in range(12):
    train = train.append(Oversampling2000)
'''
for i in range(5):
    train = train.append(Oversampling1000)
for j in range(8):
    train = train.append(Oversampling1500)
for k in range(10):
    train = train.append(Oversampling2000)
    
train = train[train['label'].notnull()]

('Number of normal transactions: ', 1089)
('Number of fraud transactions: ', 1089)
('Total number of transactions in resampled data: ', 2178)


In [15]:
train.shape

(15264, 432)

In [16]:
from sklearn import preprocessing

r_scale=preprocessing.RobustScaler()

train_scaled=r_scale.fit_transform(train[predictors])
test_scaled=r_scale.transform(test[predictors])

In [17]:
from sklearn.metrics import f1_score

In [29]:
clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])

f1_macro(y_test,result)

0.031658650391196586

In [56]:
def f1_macro0_drop(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        if sum(df['subsidy_x'] == i)!=0:
            r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
        else: 
            r=0
        if sum(df['subsidy_y'] == i)!=0:
            p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
        else:
            p=0
            
        if (r+p)!=0:
            f = r*p*2/(r+p)
        else:
            f=0
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    
    return s

In [57]:
from sklearn.metrics import make_scorer

my_score=make_scorer(score_func=f1_macro0_drop)

In [58]:
# 参数调节

from sklearn import cross_validation, metrics  
from sklearn.model_selection import GridSearchCV 

def modelfit(alg, dtrain, predictors, target,performCV=True, printFeatureImportance=False, cv_folds=3):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=cv_folds, scoring=my_score)

    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions)
    
    if performCV:
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))

    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')



In [50]:
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, under_sample_data, predictors,target)


Model Report
Accuracy : 0.8563
CV Score : Mean - 0.1463062 | Std - 0.01041427 | Min - 0.1318068 | Max - 0.1557945


In [63]:
from sklearn.pipeline import make_pipeline

In [67]:
def Oversampling(train):
    # Oversample
    Oversampling1000 = train.loc[train.label == 1000]
    Oversampling1500 = train.loc[train.label == 1500]
    Oversampling2000 = train.loc[train.label == 2000]

    for i in range(5):
        train = train.append(Oversampling1000)
    for j in range(8):
        train = train.append(Oversampling1500)
    for k in range(10):
        train = train.append(Oversampling2000)
    
    train = train[train['label'].notnull()]

In [53]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=80,min_samples_leaf=8,max_depth=3,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring=my_score,n_jobs=4,iid=False, cv=5)
#gsearch1.fit(train[predictors],train[target])

gsearch1.fit(X_undersample[predictors],y_undersample[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/kuhung/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.12288, std: 0.01573, params: {'n_estimators': 20},
  mean: 0.12506, std: 0.00925, params: {'n_estimators': 30},
  mean: 0.13755, std: 0.01015, params: {'n_estimators': 40},
  mean: 0.13984, std: 0.01478, params: {'n_estimators': 50},
  mean: 0.13868, std: 0.01559, params: {'n_estimators': 60},
  mean: 0.14296, std: 0.01391, params: {'n_estimators': 70},
  mean: 0.14423, std: 0.01627, params: {'n_estimators': 80}],
 {'n_estimators': 80},
 0.14422876456755157)

In [62]:
xgb0 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42,base_score=2000)
modelfit(xgb0, train_test, predictors,target)


Model Report
Accuracy : 0.8655
CV Score : Mean - 0.001894665 | Std - 0.00378933 | Min - 0 | Max - 0.009473324


In [76]:
rf0=RandomForestClassifier()
modelfit(rf0, train_test, predictors,target)


Model Report
Accuracy : 0.985
CV Score : Mean - 0.002591079 | Std - 0.004933227 | Min - 0 | Max - 0.01244976


In [78]:
rf0=RandomForestClassifier(class_weight='balanced')
modelfit(rf0, train_test, predictors,target)


Model Report
Accuracy : 0.987
CV Score : Mean - 0.003399415 | Std - 0.005718429 | Min - 0 | Max - 0.01478254


In [ ]:
## why the score of it is higher?
## Because of resample.After resample,the dis of dataset had changed.
# 0.02779836483035414

xgb0 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
modelfit(xgb0, under_sample_data, predictors,target)

param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=80,min_samples_leaf=8,max_depth=3,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring=my_score,n_jobs=4,iid=False, cv=5)
#gsearch1.fit(train[predictors],train[target])
gsearch1.fit(X_undersample,y_undersample)

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


In [ ]:
## Feature importance
'''
clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train[predictors],train[target])
result = clf.predict_proba(test[predictors])
f1_macro(y_test,result)

feat_imp=pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
feat_imp.tail(50)

'''

In [ ]:
'''
## Evaluation
def score(df):
    # df有三列，ID:学生ID,subsidy_x:实际奖学金金额,subsidy_y:预测奖学金金额
    correct = test_result[test_result['subsidy_x'] == test_result['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        r = float(sum(correct['subsidy_y'] == i))/sum(test_result['subsidy_x'] == i)
        p = float(sum(correct['subsidy_y'] == i))/sum(test_result['subsidy_y'] == i)
        f = r*p*2/(r+p)
        if not np.isnan(f):
            s += (float(sum(test_result['subsidy_x'] == i))/test_result.shape[0])*f
    print(s)

test_result = pd.DataFrame(columns=["studentid","subsidy_x","subsidy_y"])
test_result.studentid = ids

test_result.subsidy_x =np.array(y_test)
test_result.subsidy_y = result
test_result.subsidy_y = test_result.subsidy_y.apply(lambda x:int(x))

score(test_result)

'''